#### Подготовка данных

Изначальный датасет довольно избыточный, поэтому для проведения задания мы пользуемся этим ноутбуком, чтобы взять только нужную часть

Скачать весь датасет (~5 Гб) можно по этой ссылке: https://drive.google.com/file/d/1CUZnBtYwifVXS21yVg62T-vrPVayso5H/view

Файл с отсутствующими скелетами можно скачать из репозитория либо с google диска wget: https://github.com/shahroudy/NTURGB-D/blob/master/Matlab/NTU_RGBD_samples_with_missing_skeletons.txt

In [ ]:
!gdown 'https://drive.google.com/uc?export=dowloa&id=1CUZnBtYwifVXS21yVg62T-vrPVayso5H'

Downloading...
From: https://drive.google.com/uc?export=dowloa&id=1CUZnBtYwifVXS21yVg62T-vrPVayso5H
To: /content/nturgbd_skeletons_s001_to_s017.zip
6.18GB [02:04, 49.8MB/s]


In [ ]:
!unzip /content/nturgbd_skeletons_s001_to_s017.zip

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: nturgb+d_skeletons/S004C002P003R002A043.skeleton  
  inflating: nturgb+d_skeletons/S011C001P025R002A017.skeleton  
  inflating: nturgb+d_skeletons/S006C001P008R002A022.skeleton  
  inflating: nturgb+d_skeletons/S007C001P019R001A001.skeleton  
  inflating: nturgb+d_skeletons/S015C001P016R002A037.skeleton  
  inflating: nturgb+d_skeletons/S007C001P019R002A005.skeleton  
  inflating: nturgb+d_skeletons/S008C002P007R002A058.skeleton  
  inflating: nturgb+d_skeletons/S002C001P010R002A012.skeleton  
  inflating: nturgb+d_skeletons/S006C003P008R002A019.skeleton  
  inflating: nturgb+d_skeletons/S014C003P039R002A013.skeleton  
  inflating: nturgb+d_skeletons/S016C001P008R002A030.skeleton  
  inflating: nturgb+d_skeletons/S004C001P003R002A035.skeleton  
  inflating: nturgb+d_skeletons/S009C003P008R001A049.skeleton  
  inflating: nturgb+d_skeletons/S014C001P015R002A008.skeleton  
  inflating: nturgb+d_skeletons/S006

In [ ]:
!wget 'https://drive.google.com/uc?export=download&id=1BTN5ytqfKi71ZX6xG-ve5ETG89h0Bm1g' -O NTU_RGBD_samples_with_missing_skeletons.txt

--2021-06-24 05:39:16--  https://drive.google.com/uc?export=download&id=1BTN5ytqfKi71ZX6xG-ve5ETG89h0Bm1g
Resolving drive.google.com (drive.google.com)... 74.125.195.113, 74.125.195.100, 74.125.195.138, ...
Connecting to drive.google.com (drive.google.com)|74.125.195.113|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-04-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/qnnqsf0fqskpltmr45fnl7klv5tqpth0/1624513125000/14904333240138417226/*/1BTN5ytqfKi71ZX6xG-ve5ETG89h0Bm1g?e=download [following]
--2021-06-24 05:39:16--  https://doc-04-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/qnnqsf0fqskpltmr45fnl7klv5tqpth0/1624513125000/14904333240138417226/*/1BTN5ytqfKi71ZX6xG-ve5ETG89h0Bm1g?e=download
Resolving doc-04-c0-docs.googleusercontent.com (doc-04-c0-docs.googleusercontent.com)... 74.125.20.132, 2607:f8b0:400e:c07::84
Connecting to doc-04-c0-docs.googleusercontent.com (doc-04-

In [ ]:
data_path = "/content/nturgb+d_skeletons/"
# список отсутсвующих элементов
broken_files_path = "/content/NTU_RGBD_samples_with_missing_skeletons.txt"

In [ ]:
training_classes = [8, 10, 22, 23, 27] # классы которые будем использовать 
# для обучения, полный список представлен тут https://github.com/shahroudy/NTURGB-D

training_cameras = [1, 2, 3] 

amount_of_samples = 120
num_joint = 25
max_frame = 45  # Длина отрезка которую мы достаем из датасета

Есть функция, которая позволяет нам считать данные для каждого класса. 

In [ ]:
def read_data(data_path, broken_files_path):
    labels = []
    files = []
    action_classes = {}
    counter = 0
    files_counter = {}
              
    with open(broken_files_path, 'r') as f:
        broken_files = f.read().split("\n")

    raw_files = os.listdir(data_path)
    num_frames = 0

    for filename in raw_files:
        if filename not in broken_files:
            action_class = int(filename[filename.find('A') + 1:filename.find('A') + 4])
            camera_id = int(filename[filename.find('C') + 1:filename.find('C') + 4])
            
            if action_class in training_classes and camera_id in training_cameras:
                if action_class in action_classes:
                    if files_counter[action_class] < amount_of_samples:
                        files.append([filename, action_classes[action_class]])
                        files_counter[action_class] = files_counter[action_class] + 1
                else:
                    action_classes.update({action_class : counter})
                    files_counter.update({action_class : 1})
                    counter += 1
                    files.append([filename, action_classes[action_class]])

    print("action classes: ", action_classes)
    print("action files: ", files_counter)
    
    return files, action_classes

In [ ]:
# Функция фильтр для того, что бы найти только координаты x, y, z (т.к. в датасете
# хранится на порядок больше информации, нежели только координаты x,y,z)
# (Остальные данные нам и не нужны, т.к. у нас нет ик-камер)
def read_skeleton_filter(file):
    with open(file, 'r') as f:
        skeleton_sequence = {}
        skeleton_sequence['numFrame'] = int(f.readline())
        skeleton_sequence['frameInfo'] = []

        for t in range(skeleton_sequence['numFrame']):
            frame_info = {}
            frame_info['numBody'] = int(f.readline())
            frame_info['bodyInfo'] = []

            for m in range(frame_info['numBody']):
                body_info = {}
                body_info_key = [
                    'bodyID', 'clipedEdges', 'handLeftConfidence',
                    'handLeftState', 'handRightConfidence', 'handRightState',
                    'isResticted', 'leanX', 'leanY', 'trackingState'
                ]
                body_info = {
                    k: float(v)
                    for k, v in zip(body_info_key, f.readline().split())
                }
                body_info['numJoint'] = int(f.readline())
                body_info['jointInfo'] = []
                for v in range(body_info['numJoint']):
                    joint_info_key = [
                        'x', 'y', 'z', 'depthX', 'depthY', 'colorX', 'colorY',
                        'orientationW', 'orientationX', 'orientationY',
                        'orientationZ', 'trackingState'
                    ]
                    joint_info = {
                        k: float(v)
                        for k, v in zip(joint_info_key, f.readline().split())
                    }
                    body_info['jointInfo'].append(joint_info)
                frame_info['bodyInfo'].append(body_info)
            skeleton_sequence['frameInfo'].append(frame_info)

    return skeleton_sequence

def read_xyz(file, max_body=1, num_joint=25):
    seq_info = read_skeleton_filter(file)
    data = np.zeros((max_body, seq_info['numFrame'], num_joint, 3))
    for n, f in enumerate(seq_info['frameInfo']):
        for m, b in enumerate(f['bodyInfo']):
            for j, v in enumerate(b['jointInfo']):
                if m < max_body and j < num_joint:
                    data[m, n, j, :] = [v['x'], v['y'], v['z']]

    return data

**В этой функции меняем количество фреймов подаваемых на вход модели**

В задании нужно пронаблюдать зависимость качества обучения модели от количества кадров, которые мы подаем в модель. Т.е. то, cколько кадров у нас отведено на один блок.

По хорошему функцию ниже нужно прописать внутри определения датасета.

In [ ]:
def create_coords_blocks(data, chunk_len=45):   
    frame_counter = 0
    new_labels = []
    new_frames = []
    blocks = []
    
    test_frames = read_xyz(data_path + data[0])[0]
    label = data[1]
    slice_len = chunk_len * int(len(test_frames)/chunk_len)


    for index in range(len(test_frames[:slice_len])):
        frame_counter += 1
        new_frames.append(test_frames[index].flatten())
        if frame_counter == chunk_len:
            frame_counter = 0
            blocks.append(np.array(new_frames))
            new_labels = new_labels + [label]
            new_frames = []
       
            
    return blocks, new_labels

In [ ]:
# список файлов с лейблами на каждый файл 
import os
import numpy as np
import pandas as pd


working_files_with_labels, action_classes = read_data(data_path, broken_files_path)

action classes:  {27: 0, 10: 1, 8: 2, 22: 3, 23: 4}
action files:  {27: 120, 10: 120, 8: 120, 22: 120, 23: 120}


In [ ]:
LABELS = {v: k for k, v in action_classes.items()}

Здесь выносится сгенерированный список лейблов с номером класса.

In [ ]:
LABELS

{0: 27, 1: 10, 2: 8, 3: 22, 4: 23}

In [ ]:
working_files_with_labels[:10]

[['S008C001P035R001A027.skeleton', 0],
 ['S007C003P019R001A010.skeleton', 1],
 ['S012C001P015R002A008.skeleton', 2],
 ['S010C001P008R002A027.skeleton', 0],
 ['S004C002P007R001A022.skeleton', 3],
 ['S007C003P026R001A023.skeleton', 4],
 ['S013C001P019R001A010.skeleton', 1],
 ['S001C003P004R002A010.skeleton', 1],
 ['S012C002P025R002A008.skeleton', 2],
 ['S008C001P029R001A010.skeleton', 1]]

In [ ]:
len(working_files_with_labels)

600

In [ ]:
data = []
labels = []
numbers = {0: 0, 1 : 0, 2 : 0, 3 : 0, 4 :0}
for file_label in working_files_with_labels:
    frames_blocks, label = create_coords_blocks(file_label, chunk_len=max_frame)
    
    if label != [] and numbers[label[0]] < amount_of_samples:
        numbers[label[0]] = numbers[label[0]] + len(label)
        data = data + frames_blocks
        labels = labels + label


data_np = np.asarray(data)
labels_np = np.asarray(labels)

data_sq = data_np.reshape(len(data_np), -1)
test_data = pd.DataFrame(data_sq)
test_labels = pd.DataFrame(labels_np)
test_data['labels'] = test_labels

In [ ]:
numbers

{0: 117, 1: 120, 2: 120, 3: 120, 4: 120}

In [ ]:
test_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,3336,3337,3338,3339,3340,3341,3342,3343,3344,3345,3346,3347,3348,3349,3350,3351,3352,3353,3354,3355,3356,3357,3358,3359,3360,3361,3362,3363,3364,3365,3366,3367,3368,3369,3370,3371,3372,3373,3374,labels
0,-0.262893,-0.508769,3.602156,-0.239165,-0.189954,3.597205,-0.215966,0.123259,3.577788,-0.264699,0.253853,3.587672,-0.381152,-0.026487,3.707927,-0.397388,-0.291071,3.717805,-0.437680,-0.518163,3.653854,-0.430419,-0.557180,3.638437,-0.108027,-0.001286,3.466215,-0.058958,-0.250904,3.427291,-0.109327,-0.474664,3.354377,-0.138881,-0.535095,3.337629,-0.317090,-0.506285,3.609146,-0.316495,...,-0.334767,-0.408200,3.558891,-0.330124,-0.799601,3.571123,-0.266874,-1.170104,3.650408,-0.283914,-1.250417,3.571604,-0.215881,-0.392205,3.492639,-0.173510,-0.773299,3.440047,-0.138333,-1.156085,3.477470,-0.166955,-1.238683,3.413666,-0.309296,0.159415,3.505794,-0.476424,-0.414646,3.626238,-0.453931,-0.396062,3.627500,-0.105399,-0.427268,3.351365,-0.143089,-0.328721,3.369715,0
1,-0.076705,-0.421223,3.535730,-0.084606,-0.158787,3.616801,-0.092298,0.101431,3.678699,-0.086509,0.238899,3.684279,-0.262103,-0.011662,3.642337,-0.285818,-0.221431,3.557834,-0.220482,-0.323413,3.353544,-0.165695,-0.328161,3.302737,0.078263,-0.018137,3.622106,0.120910,-0.217440,3.548152,0.034311,-0.282915,3.356349,-0.022333,-0.313811,3.268240,-0.159619,-0.417924,3.500591,-0.282068,...,-0.164002,-0.420377,3.552311,-0.289378,-0.378400,3.170420,-0.233107,-0.751483,3.201664,-0.227435,-0.833191,3.119594,-0.003543,-0.413005,3.539908,0.023846,-0.388371,3.141783,-0.018057,-0.747195,3.214643,-0.027027,-0.777730,3.066576,-0.091075,0.036853,3.672379,-0.205453,-0.336388,3.295616,-0.204604,-0.334998,3.282000,-0.103943,-0.032425,3.285205,-0.087397,-0.046391,3.303000,1
2,0.236271,-0.248975,3.470248,0.206922,0.020067,3.400594,0.177815,0.285034,3.319195,0.190815,0.402819,3.305032,0.111557,0.155606,3.257149,0.061049,-0.054604,3.288778,0.106004,-0.268230,3.301626,0.108967,-0.303941,3.311796,0.302040,0.211207,3.387733,0.352206,0.107865,3.487331,0.280520,0.255111,3.331321,0.254470,0.297555,3.302099,0.187617,-0.251805,3.417877,0.175226,...,0.023089,-0.486613,3.678008,0.302037,-0.579365,3.532736,0.156052,-0.839332,3.644817,0.185073,-0.899975,3.551588,0.139441,-0.481252,3.709099,-0.125906,-0.342558,3.783519,0.118890,-0.613639,3.690825,0.148213,-0.676113,3.597222,0.112421,-0.053781,3.523969,-0.118740,-0.530832,3.466256,-0.124346,-0.513340,3.432429,0.300113,-0.426531,3.809955,0.254244,-0.386234,3.829200,2
3,0.409361,-0.223792,2.800204,0.405338,0.069001,2.717275,0.404676,0.353825,2.620399,0.462238,0.485232,2.605532,0.283753,0.228882,2.560761,0.216874,-0.004651,2.584627,0.236054,-0.222737,2.603074,0.231503,-0.273601,2.621502,0.544713,0.251158,2.731055,0.555448,0.000884,2.842023,0.553088,-0.219667,2.843848,0.568906,-0.245808,2.852252,0.344362,-0.221405,2.734210,0.272189,...,0.474123,-0.451515,2.647996,0.408240,-0.704604,2.564457,0.421207,-0.915678,2.747169,0.345083,-0.979703,2.672730,0.548514,-0.440742,2.561994,0.622209,-0.690650,2.538265,0.579772,-0.922380,2.758182,0.521800,-0.950909,2.795265,0.361655,0.032189,2.470701,0.365360,-0.541136,2.707641,0.364287,-0.510073,2.699000,0.210243,-0.409262,2.465283,0.222766,-0.360625,2.431706,0
4,0.182411,-0.451482,2.985908,0.274911,-0.141932,3.008283,0.356072,0.169988,3.019586,0.352905,0.279639,3.061612,0.200502,-0.004270,2.995412,0.098091,-0.271342,2.867519,-0.193955,-0.285940,2.938804,-0.268820,-0.290788,2.953335,0.391154,0.047917,2.884517,0.313775,-0.188912,2.743200,0.088939,-0.252958,2.715916,0.031348,-0.265601,2.742607,0.121323,-0.437438,2.954270,-0.232830,...,0.305139,-0.288980,2.947938,-0.174408,-0.218393,3.034641,-0.300802,-0.651076,3.000658,-0.317810,-0.757216,2.945637,0.297451,-0.216681,2.800683,0.151259,0.054777,2.961278,-0.039097,-0.173355,3.285156,0.042549,-0.238918,3.160681,-0.080621,-0.286304,3.054650,-0.404052,-0.591078,2.731566,-0.3974

**Ваш итоговый файл**

In [ ]:
test_data.to_csv("new_skels.csv", index=False)